## Experimentação de modelos
 Nesta sessão irei realizar uma série de experimentos, combinando diferentes modelos e preprocessamentos. Para registrar tudo isso, irei utilizar o MLFlow.

Aqui irei testar modelos baseados em distância e árvore, segue os esboços dos testes a serem realizados:

Baseados em distância:
- Standard Scaler em todas as variáveis numéricas + One Hot Encoder em todas as categóricas.
- Standard Scaler em todas as variáveis numéricas + One Hot Encoder nas categóricas nominais + Ordinal Encoder nas ordinais.
- Standard Scaler nas variáveis numéricas sem outliers + Robust Scaler nas variáveis com outliers + One Hot Encoder em todas as categóricas.
- Standard Scaler nas variáveis numéricas sem outliers + Robust Scaler nas variáveis com outliers + One Hot Encoder em nas categóricas nominais + Ordinal Encoder nas ordinais.
- Standard Scaler em todas as variáveis numéricas + One Hot Encoder em todas as categóricas + Isolation Forest para rotular os outliers.
- Standard Scaler em todas as variáveis numéricas + One Hot Encoder nas categóricas nominais + Ordinal Encoder nas ordinais + Isolation Forest para rotular os outliers.
- Standard Scaler nas variáveis numéricas sem outliers + Robust Scaler nas variáveis com outliers + One Hot Encoder em todas as categóricas + Isolation Forest para rotular os outliers.
- Standard Scaler nas variáveis numéricas sem outliers + Robust Scaler nas variáveis com outliers + One Hot Encoder em nas categóricas nominais + Ordinal Encoder nas ordinais + Isolation Forest para rotular os outliers.


Baseados em árvore:
- Diferentes tipos de encoders da biblioteca **category encoders**  para as categóricas
- Diferentes tipos de encoders da biblioteca **category encoders**  para as categóricas + Isolation Forest para rotular os outliers.
Geral:

In [34]:
# Importando as bibliotecas

import category_encoders as ce
import mlflow
import numpy as np
import pandas as pd

# Pré-processamento
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler
from sklearn.ensemble import IsolationForest

# Modelos
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, SGDRegressor
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Metricas
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [23]:
def registrarexperimento(modelos, tags, transformer, x_treino, y_treino):
    """
    Função para registrar experimentos de acordo com modelos, tags e transformadores específicos.
    :param modelos: lista contendo os modelos a serem testados
    :param tags: lista contendo tags de identificação para cada modelo e transformações aplicadas
    :param transformer: transformações que serão aplicadas ao conjunto de treino
    :param x_treino: dados contendo as variáveis dependentes
    :param y_treino: target
    :return: None
    """
    # Iterando com os modelos da lista
    for model in modelos:

        # Iniciando os experimentos
        with mlflow.start_run():

            # Colocando uma tag para identificação
            index_modelo = modelos.index(model)
            mlflow.set_tag('modelo', tags[index_modelo])

            # Criando o pipeline
            pipe = Pipeline([('transformer', transformer),
                             ('regressor', model)])

            # Calculando as métricas com cross-validation
            mae_mean = cross_val_score(pipe,
                                        x_treino,
                                        y_treino,
                                        cv=5,
                                        scoring='neg_mean_absolute_error').mean() * (-1)

            mae_std = cross_val_score(pipe,
                                        x_treino,
                                        y_treino,
                                        cv=5,
                                        scoring='neg_mean_absolute_error').std() * (-1)

            rmse_mean = cross_val_score(pipe,
                                        x_treino,
                                        y_treino,
                                        cv=5,
                                        scoring='neg_root_mean_squared_error').mean() * (-1)

            rmse_std = cross_val_score(pipe,
                                        x_treino,
                                        y_treino,
                                        cv=5,
                                        scoring='neg_root_mean_squared_error').std() * (-1)

            # Salvando as métricas
            mlflow.log_metric('mae_mean', mae_mean)
            mlflow.log_metric('mae_std', mae_std)
            mlflow.log_metric('rmse_mean', rmse_mean)
            mlflow.log_metric('rmse_std', rmse_std)

    return None

In [2]:
# Importando os dados
dados = pd.read_csv('../data/interim/dados_para_treino.csv', index_col=0)

In [3]:
# Checando a tabela
dados.head()

,category,customer_type,unit_price,quantity,total,payment_type,prcnt_stock,mean,median,min,max,day_of_week,is_weekend,hour,turn
0,fruit,gold,3.99,2,7.98,e-wallet,0.23,-1.05,-2.24,-2.89,1.67,2,no,9,morning
1,fruit,premium,1.49,1,1.49,credit card,0.54,-1.05,-2.24,-2.89,1.67,2,no,9,morning
2,fruit,non-member,4.49,4,17.96,credit card,0.71,-1.05,-2.24,-2.89,1.67,2,no,9,morning
3,refrigerated items,gold,5.99,1,5.99,credit card,0.54,-1.05,-2.24,-2.89,1.67,2,no,9,morning
4,canned foods,gold,2.49,1,2.49,e-wallet,0.33,-1.05,-2.24,-2.89,1.67,2,no,9,morning


### Alterando o tipo das colunas
Antes de iniciar os experimentos irei realizar uma pequena mudança nos tipos das colunas.

In [10]:
# Alterando o tipo de dado
dados['day_of_week'] = dados.day_of_week.astype('object')
dados['customer_type'] = dados.customer_type.astype('category')

## Buscando o melhor algorítmo

A partir daqui iniciarei os experimentos. Nesta etapa **NÃO** irei alterar os parâmetros, onde o melhor modelo irá receber tunning em outra rodada de experimentos.

Iniciarei os experimentos com os algoritmos **baseados em distância**.

In [5]:
# Define o local para salvar os exoerimentos
mlflow.set_tracking_uri('../mlruns')

# Criando/acessando o experimento
mlflow.set_experiment('Comparando modelos')

# Dividindo os dados em variáveis dependentes e independentes
x = dados.drop(columns='prcnt_stock')
y = dados.prcnt_stock

# Dividindo os dados em treino e teste
x_treino, x_teste, y_treino, y_teste = train_test_split(x,
                                                        y,
                                                        test_size=0.25,
                                                        random_state=14)

# Instanciando os modelos
linear_reg = LinearRegression()
lasso = Lasso()
ridge = Ridge()
elastic_nt = ElasticNet()
reg_estocastico = SGDRegressor()

# Criando listas com os modelo
modelos = [linear_reg, lasso, ridge, elastic_nt, reg_estocastico]

### Standard Scaler em todas as variáveis numéricas + One Hot Encoder em todas as categóricas.

In [11]:
# Instanciando os transformadores
sc = StandardScaler()
ohe = OneHotEncoder(drop='first')

# Selecionando os dados por tipo
numericas = x_treino.select_dtypes(['int', 'float']).columns
categoricas = x_treino.select_dtypes(['object', 'category']).columns

# Criando o transformer
transformer = ColumnTransformer(transformers=[('scaler', sc, numericas),
                                              ('encoder', ohe, categoricas)])

# Criando lista com as tags para identificar modelo e preprocessamento usado
tags = ['Reg_Linear_SC_OHE', 'Lasso_SC_OHE', 'Ridge_SC_OHE', 'Elastic_Net_SC_OHE', 'Reg_Estocástico_SC_OHE']

In [22]:
# Criando os experimentos
registrarexperimento(modelos, tags, transformer, x_treino, y_treino)

### Standard Scaler em todas as variáveis numéricas + One Hot Encoder nas categóricas nominais + Ordinal Encoder nas ordinais

In [18]:
# Instanciando os transformadores
sc = StandardScaler()
ohe = OneHotEncoder(drop='first')
oe = ce.ordinal.OrdinalEncoder()

# Selecionando os dados por tipo
numericas = x_treino.select_dtypes(['int', 'float']).columns
categoricas = x_treino.select_dtypes('object').columns
ordinais = x_treino.select_dtypes('category').columns

# Criando o transformer
transformer = ColumnTransformer(transformers=[('scaler', sc, numericas),
                                              ('encoder_nominal', ohe, categoricas),
                                              ('encoder_ordinal', oe, ordinais)])

# Criando lista com as tags para identificar modelo e preprocessamento usado
tags = ['Reg_Linear_SC_OHE_OE', 'Lasso_SC_OHE_OE', 'Ridge_SC_OHE_OE', 'Elastic_Net_SC_OHE_OE', 'Reg_Estocástico_SC_OHE_OE']

In [19]:
# Criando os experimentos
registrarexperimento(modelos, tags, transformer, x_treino, y_treino)

### Standard Scaler nas variáveis numéricas sem outliers + Robust Scaler nas variáveis com outliers + One Hot Encoder em todas as categóricas.


In [29]:
# Criando uma lista vazia
col_com_outlier = []

# Buscando todas as variáveis numéricas
numericas = x_treino.select_dtypes(['int', 'float']).columns

# Iterando sobre as colunas numéricas
for col in numericas:

    # Calculando os quartis e o IQR
    q1 = np.quantile(dados[col], 0.25)
    q3 = np.quantile(dados[col], 0.75)
    iqr = q3 - q1

    #Separando os dados sem e com outliers
    if dados.query(f"{col} > {q3 + 1.5 * iqr}").shape[0] > 0:
        col_com_outlier.append(col)

In [33]:
# Instanciando os transformadores
sc = StandardScaler()
rs = RobustScaler()
ohe = OneHotEncoder(drop='first')

# Selecionando os dados por tipo
num_com_outliers = col_com_outlier
num_sem_outliers = [col for col in x_treino.select_dtypes(['int', 'float']).columns if col not in col_com_outlier]
categoricas = x_treino.select_dtypes(['object', 'category']).columns

# Criando o transformer
transformer = ColumnTransformer(transformers=[('scaler', sc, num_sem_outliers),
                                              ('scaler_outliers', rs, num_com_outliers),
                                              ('encoder_nominal', ohe, categoricas)])

# Criando lista com as tags para identificar modelo e preprocessamento usado
tags = ['Reg_Linear_SC_RS_OHE', 'Lasso_SC_RS_OHE', 'Ridge_SC_RS_OHE', 'Elastic_Net_SC_RS_OHE', 'Reg_Estocástico_SC_RS_OHE']

In [ ]:
# Criando os experimentos
registrarexperimento(modelos, tags, transformer, x_treino, y_treino)

### Standard Scaler nas variáveis numéricas sem outliers + Robust Scaler nas variáveis com outliers + One Hot Encoder em nas categóricas nominais + Ordinal Encoder nas ordinais

In [ ]:
# Criando uma lista vazia
col_com_outlier = []

# Buscando todas as variáveis numéricas
numericas = x_treino.select_dtypes(['int', 'float']).columns

# Iterando sobre as colunas numéricas
for col in numericas:

    # Calculando os quartis e o IQR
    q1 = np.quantile(dados[col], 0.25)
    q3 = np.quantile(dados[col], 0.75)
    iqr = q3 - q1

    #Separando os dados sem e com outliers
    if dados.query(f"{col} > {q3 + 1.5 * iqr}").shape[0] > 0:
        col_com_outlier.append(col)

In [ ]:
# Instanciando os transformadores
sc = StandardScaler()
rs = RobustScaler()
ohe = OneHotEncoder(drop='first')
oe = ce.ordinal.OrdinalEncoder()

# Selecionando os dados por tipo
num_com_outliers = col_com_outlier
num_sem_outliers = [col for col in x_treino.select_dtypes(['int', 'float']).columns if col not in col_com_outlier]
categoricas = x_treino.select_dtypes('object').columns
ordinais = x_treino.select_dtypes('category').columns

# Criando o transformer
transformer = ColumnTransformer(transformers=[('scaler', sc, num_sem_outliers),
                                              ('scaler_outliers', rs, num_com_outliers),
                                              ('encoder_nominal', ohe, categoricas),
                                              ('encoder_ordinal', oe, ordinais)])

# Criando lista com as tags para identificar modelo e preprocessamento usado
tags = ['Reg_Linear_SC_RS_OHE_OE', 'Lasso_SC_RS_OHE_OE', 'Ridge_SC_RS_OHE_OE', 'Elastic_Net_SC_RS_OHE_OE', 'Reg_Estocástico_SC_RS_OHE_OE']

In [ ]:
# Criando os experimentos
registrarexperimento(modelos, tags, transformer, x_treino, y_treino)

### Standard Scaler em todas as variáveis numéricas + One Hot Encoder em todas as categóricas + Isolation Forest para rotular os outliers

In [ ]:
# Instanciando os transformadores
sc = StandardScaler()
ohe = OneHotEncoder(drop='first')

# Selecionando os dados por tipo
numericas = x_treino.select_dtypes(['int', 'float']).columns
categoricas = x_treino.select_dtypes(['object', 'category']).columns

# Criando o transformer
transformer = ColumnTransformer(transformers=[('scaler', sc, numericas),
                                              ('encoder', ohe, categoricas)])

# Criando lista com as tags para identificar modelo e preprocessamento usado
tags = ['Reg_Linear_SC_OHE_IF', 'Lasso_SC_OHE_IF', 'Ridge_SC_OHE_IF', 'Elastic_Net_SC_OHE_IF', 'Reg_Estocástico_SC_OHE_IF']